In [1]:
import sys

sys.path.append("../../")
import pandas as pd
import numpy as np
import datetime
import os
from pprint import pprint
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 20)

plt.style.use("classic")
plt.rcParams["axes.grid"] = True
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["axes.formatter.useoffset"] = False
plt.rcParams["axes.formatter.limits"] = [-1000000000, 1000000000]

In [2]:
base_data_path = "../../phitech-data/"
os.listdir(f"{base_data_path}/01_raw")

['6B_300d_1min.csv']

In [4]:
res = pd.read_csv(f"{base_data_path}/01_raw/6B_300d_1min.csv")
res.columns = [
    "date",
    "time",
    "open",
    "high",
    "low",
    "close",
    "volume",
    "#_of_trades",
    "ohlc_avg",
    "hlc_avg",
    "hl_avg",
    "bid_volume",
    "ask_volume",
    "avg",
    "avg_1",
]
res.head()

,date,time,open,high,low,close,volume,#_of_trades,ohlc_avg,hlc_avg,hl_avg,bid_volume,ask_volume,avg,avg_1
0,2024-5-15,21:00:00.000000,1.2690,1.2691,1.2689,1.2690,281,85,1.2690,1.2690,1.2690,97,184,1.2690,1.2690
1,2024-5-15,21:05:00.000000,1.2689,1.2689,1.2689,1.2689,70,22,1.2689,1.2689,1.2689,56,14,1.2689,1.2689
2,2024-5-15,21:10:00.000000,1.2689,1.2690,1.2688,1.2690,60,20,1.2689,1.2689,1.2689,17,43,1.2690,1.2690
3,2024-5-15,21:15:00.000000,1.2690,1.2691,1.2690,1.2691,136,34,1.2690,1.2691,1.2690,61,75,1.2690,1.2690
4,2024-5-15,21:20:00.000000,1.2691,1.2693,1.2691,1.2692,234,67,1.2692,1.2692,1.2692,116,118,1.2690,1.2690


In [25]:
long_signals = pd.Series([1 if x % 10 == 0 else 0 for x in range(res.shape[0])])
short_signals = pd.Series([1 if x % 15 == 0 else 0 for x in range(res.shape[0])])
long_signals.value_counts()

0    52594
1     5844
Name: count, dtype: int64

In [26]:
pf = vbt.Portfolio.from_signals(res.close, long_signals, short_signals, freq=10, fees=1)
pf

In [94]:
ma20 = vbt.MA.run(res.close, window=20)
ma120 = vbt.MA.run(res.close, window=120)

entries = ma20.ma_crossed_below(ma120)
exits = ma20.ma_crossed_above(ma120)

In [92]:
ores = vbt.Portfolio.from_signals(res.close, entries, exits, freq=60)
ores.stats()

Start                                                 0
End                                               58437
Period                        0 days 00:00:00.003506280
Start Value                                       100.0
End Value                                    105.737479
Total Return [%]                               5.737479
Benchmark Return [%]                           1.962175
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                               5.528331
Max Drawdown Duration         0 days 00:00:00.001828500
Total Trades                                        352
Total Closed Trades                                 352
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  69.034091
Best Trade [%]                                 0.977975
Worst Trade [%]                               -1

In [81]:
ores.plot_cum_returns()

FigureWidget({
    'data': [{'line': {'color': '#7f7f7f'},
              'name': 'Benchmark',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'de7a6e8e-9b6e-4c04-8a6b-41eca5a44771',
              'x': array([    0,     1,     2, ..., 58435, 58436, 58437]),
              'y': array([1.        , 0.9999212 , 1.        , ..., 1.01993696, 1.01970055,
                          1.01962175])},
             {'hoverinfo': 'skip',
              'line': {'color': 'rgba(0, 0, 0, 0)', 'width': 0},
              'opacity': 0,
              'showlegend': False,
              'type': 'scatter',
              'uid': '59176430-b7da-44de-9ea4-b2f7ba238420',
              'x': array([    0,     1,     2, ..., 58435, 58436, 58437]),
              'y': array([1, 1, 1, ..., 1, 1, 1])},
             {'connectgaps': False,
              'fill': 'tonexty',
              'fillcolor': 'rgba(0, 128, 0, 0.3)',
              'hoverinfo': 'skip',
              'line': {'color': 

In [83]:
ores.plot_drawdowns()

FigureWidget({
    'data': [{'line': {'color': '#9467bd'},
              'name': 'Value',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'c4a7d92d-0413-45a0-895e-551cb2049a0b',
              'x': array([    0,     1,     2, ..., 58435, 58436, 58437]),
              'y': array([100.       , 100.       , 100.       , ..., 105.7374792, 105.7374792,
                          105.7374792])},
             {'customdata': array([[ 40],
                                   [ 48],
                                   [ 79],
                                   [ 82],
                                   [109]]),
              'hovertemplate': 'Drawdown Id: %{customdata[0]}<br>Peak Timestamp: %{x}<br>Peak Value: %{y}',
              'marker': {'color': '#4285F4',
                         'line': {'color': 'rgb(11,84,205)', 'width': 1},
                         'size': 7,
                         'symbol': 'diamond'},
              'mode': 'markers',
              